https://ratsgo.github.io/nlpbook/docs/doc_cls/train/

https://github.com/ratsgo/ratsnlp/tree/master/ratsnlp/nlpbook

In [2]:
# /usr/local/lib/python3.10/dist-packages/ratsnlp 에 설치
!pip install ratsnlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2


# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.



학습 인자를 위한 데이터클래스: ClassificationTrainArguments

이 클래스는 분류 모델 학습과 관련된 다양한 설정을 보유하도록 설계되었습니다. dataclass의 사용은 __init__, __repr__ 등과 같은 내장 메서드를 포함한 클래스 인스턴스 생성을 단순화합니다.

속성:
- pretrained_model_name: 사용할 사전 훈련된 모델의 이름입니다.
- downstream_task_name: 조직 목적을 위한 작업의 이름입니다.
- downstream_corpus_name: 사용된 데이터의 구체적인 이름입니다.
- downstream_corpus_root_dir: 데이터가 저장된 루트 디렉토리입니다.
- downstream_model_dir: 학습된 모델이 저장될 디렉토리입니다.
- max_seq_length: 토큰화를 위한 최대 시퀀스 길이입니다.
- save_top_k: 모니터링 조건에 기반한 상위 모델 체크포인트를 저장할 개수입니다.
- monitor: 최고의 모델 체크포인트를 저장하기 위한 모니터링 조건입니다(예: 검증 손실).
- seed: 재현성을 위한 랜덤 시드입니다.
- overwrite_cache: 캐시된 학습 및 평가 세트를 덮어쓸지 여부입니다.
- force_download: 이미 존재하더라도 데이터 및 모델을 강제로 다운로드할지 여부입니다.
- test_mode: 테스트를 위한 빠른 개발 실행을 활성화합니다.
- learning_rate: 최적화를 위한 학습률입니다.
- epochs: 학습 에폭 수입니다.
- batch_size: 학습을 위한 배치 크기입니다. 0으로 설정하면 최적의 배치 크기가 자동으로 찾아집니다.
- cpu_workers: 데이터 로딩을 위한 CPU 워커의 수입니다.
- fp16: 더 빠른 계산을 위해 16비트 부동 소수점으로 학습을 활성화합니다.
- tpu_cores: 사용 가능한 경우 TPU 코어에서 학습을 활성화합니다.

In [3]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls1",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=1,

    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [4]:
# args에 지정된 시드로 고정하는 역할
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [5]:
# 각종 로그들을 출력하는 로거를 설정. utils.py
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls1', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_size=32, cpu_workers=8, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='document-classification', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-doccls1', max_seq_length=128, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=1, batch_s

# 말뭉치 다운로드
- 실습에 사용할 말뭉치(Naver Sentiment Movie review Corpus)를 다운로드합니다.
- 코랩 환경 로컬의 root_dir(/content/Korpora) 이하에 저장

In [6]:
from Korpora import Korpora
Korpora.fetch( # 정 한국어 코퍼스를 Korpora 서버에서 로컬로 다운로드하는 역할
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True, # 동일한 이름의 코퍼스가 이미 존재하는 경우에도 코퍼스를 다운로드
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 28.5MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 13.2MB/s]                            


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다.
kcbert-base 모델이 사용하는 토크나이저(tokenizer)를 선언합니다. 토크나이저(tokenizer)는 토큰화를 수행하는 프로그램이라는 뜻

In [7]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False, # 텍스트를 토큰화할 때 대소문자를 구분하여 처리
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

## 학습데이터 구축
학습데이터를 만듭니다.

- 딥러닝 모델을 학습하려면 학습데이터를 배치(batch) 단위로 지속적으로 모델에 공급해 주어야 합니다. 파이토치(PyTorch)에서는 이 역할을 데이터 로더(DataLoader)가 수행
- NsmcCorpus는 CSV 파일 형식의 NSMC 데이터를 “문장(영화 리뷰) + 레이블(긍정, 부정)” 형태로 읽어들이는 역할
- ClassificationDataset는 각 instance를 포함하는 데이터셋 역할을 수행.
ClassificationDataset 클래스를 좀 더 자세히 살펴보겠습니다. 이 클래스는 NsmcCorpus와 토크나이저를 품고 있는데요. NsmcCorpus가 넘겨준 데이터(문장, 레이블)를 모델이 학습할 수 있는 형태로 가공합니다. 다시 말해 문장을 토큰화하고 이를 인덱스로 변환하는 한편, 레이블 역시 정수(integer)로 바꿔주는 역할을 한다.
- ClassificationDataset은 input_ids, attention_mask, token_type_ids의 길이가 모두 128인 이유는 토큰 기준 최대 길이(max_seq_length)를 args에서 128로 설정해 두었기 때문입니다.
- input_ids에 패딩 토큰([PAD])의 인덱스에 해당하는 0이 많이 붙어 있음을 확인할 수 있습니다. 분석 대상 문장의 토큰 길이가 max_seq_length보다 짧아서입니다. 이보다 긴 문장일 경우 128로 줄입니다.
- attention_mask는 해당 토큰이 패딩 토큰인지(0) 아닌지(1)를 나타내며 token_type_ids는 세그먼트(segment) 정보로 기본값은 모두 0으로 넣습니다. label은 정수로 변환
- sampler는 샘플링 방식을 정의합니다. 데이터 로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 갯수만큼을 비복원(replacement=False) 랜덤 추출한다. 학습용 데이터 로더와 달리 평가용 데이터 로더는 SequentialSampler를 사용한다. SequentialSampler는 batch_size만큼의 갯수만큼을 인스턴스 순서대로 추출하는 역할을 한다. 학습 때 배치 구성은 랜덤으로 하는 것이 좋으나 평가할 때는 평가용 데이터 전체를 사용하기 때문에 굳이 랜덤으로 구성할 이유가 없기 때문에 SequentialSampler를 쓴다.
- collate_fn은 이렇게 뽑힌 인스턴스를 배치로 만드는 역할을 하는 함수


In [8]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus() #  데이터 예제를 추출하고 레이블을 관리하는 메소드를 제공
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False), # 샘플링시 중복을 허용하지 않아 미니배치는 고유한 샘플로만 구성
    collate_fn=nlpbook.data_collator,
    drop_last=False, # 배치 크키로 채우지 못한 마지막 불완전 배치의 사용 여부
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 31.826 s]
INFO:ratsnlp:tokenize sentences [took 31.826 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:sentence: 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: [CLS] 아 더 ##빙 . . 진짜 짜증나네 ##요 목소리 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [9]:
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 10.575 s]
INFO:ratsnlp:tokenize sentences [took 10.575 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:sentence: 굳 ㅋ
INFO:ratsnlp:tokens: [CLS] 굳 ㅋ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 분류를 수행할 모델을 초기화
- BertForSequenceClassification은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스

In [10]:
# pretrained_model_name="beomi/kcbert-base",
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [11]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly i

# 학습 준비
Task와 Trainer를 준비합니다.

In [12]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [13]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-doccls`)에 저장됩니다.

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

#### 데코레이터

In [ ]:
# say_hello 함수에 @my_decorator 데코레이터를 적용하면, say_hello 함수를 호출할 때마다 my_decorator에 의해 추가된 동작이 수행
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()


Something is happening before the function is called.
Hello!
Something is happening after the function is called.


In [ ]:
# "@property" 데코레이터가 적용된 메소드는 속성처럼 접근이 가능
class MyClass:
    def __init__(self):
        self._x = None

    @property
    def x(self):
        return self._x


In [ ]:
# x는 메소드가 아닌 읽기 전용의 속성처럼 동작
obj = MyClass()
print(obj.x)  # None


None


In [ ]:
# "@property" 데코레이터는 게터(getter) 역할을 합니다. 즉, 객체의 내부 상태를 외부로 전달
# "@property"에는 해당 속성을 설정하는 세터(setter) 메소드를 정의할 수 있는 기능도 있다. 세터 메소드는 "@<속성명>.setter" 데코레이터를 사용하여 정의
class MyClass:
    def __init__(self):
        self._x = None

    @property
    def x(self):
        return self._x

    @x.setter
    def x(self, value):
        self._x = value


In [ ]:
# x 속성은 값을 읽고 쓸 수 있는 속성이 된다.
obj = MyClass()
obj.x = 10  # 값을 설정
print(obj.x)  # 값을 읽음: 10


10
